# Taller de análisis de sentimientos

## Parte 1: Cargaremos las clases necesarias para poder trabajar.

In [1]:
import numpy as np
from optparse import OptionParser
import sys
import re
from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.linear_model import LogisticRegression as LR
from nltk.tokenize import TweetTokenizer
from sklearn import metrics
import random
np.random.seed(42069)
random.seed(69420)

## Parte 2: Cargamos los datos de entrenamiento y pruebas en formatos que nos pueda entender SKLearn

In [2]:
with open("CLASIFICADOS.txt", "r", encoding="utf-8") as f:
    # Creando una lista o arreglo de tuits
    x_train = re.split('\nGolden label: -*[01]\n', f.read())
    print("Tamaño del conjunto de datos: {}.\nPrimer ejemplo:\n{}".format(len(x_train), x_train[0]))
    f.seek(0)
    
    # Creando una lista o arreglo de las etiquetas que corresponden a cada tuit
    y_train = re.findall('-*[01]', ''.join(re.findall('Golden label: -*[01]', f.read())))
    print("\nTamaño de etiquetas {}\nPrimer etiqueta: {}".format(len(y_train), y_train[0]))
with open("pruebas.txt", 'r', encoding="utf-8") as f:
    x_test = re.split('\nGolden label: -*[01]\n', f.read())
    print("\nTamaño de pruebas: {}.\nPrimer ejemplo:\n{}".format(len(x_test), x_test[0]))
    f.seek(0)
    y_test = re.findall('-*[01]', ''.join(re.findall('Golden label: -*[01]', f.read())))
    print("\nTamaño de etiquetas {}\nPrimer etiqueta: {}".format(len(y_test), y_test[0]))

Tamaño del conjunto de datos: 1056.
Primer ejemplo:
Y NO EXISTE, no por un megaproyecto de algún aeropuerto hijoeputa que vino a promover un ecocidio en la "rica y verde zona del actual Texcoco", sino que todo comenzó allá por la época prehispánica, cuando los primeros chilangos tuvieron la muy rara idea de construir Tenochtitlan

Tamaño de etiquetas 1056
Primer etiqueta: 0

Tamaño de pruebas: 237.
Primer ejemplo:
#CienciaUNAM El suelo de Texcoco no sólo es el menos apto, es el peor suelo en el que podrían construir un aeropuerto; una obra de este tipo requiere un suelo más sólido,  firme y esta área es demasiado inestable para una edificación de esa magnitud:

https://t.co/rjM4HR4zll https://t.co/ayW95wqcfN

Tamaño de etiquetas 237
Primer etiqueta: 1


## Parte 3: Representación númerica de los tuits: tokenizadores y otros parámetros

In [3]:
"""
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(x_train)
print(sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]))

vectorizer = TfidfVectorizer(max_df=0.85, min_df=8)
tfidf = vectorizer.fit_transform(x_train)
sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1])
"""

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True)
vectorizer = TfidfVectorizer(max_df=0.85, min_df=8, tokenizer=tokenizer.tokenize)
tfidf = vectorizer.fit_transform(x_train)

tfidf_test = vectorizer.transform(x_test)

def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
def extract_topn_from_vector(feature_names, sorted_items, topn):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

vocab = vectorizer.get_feature_names()
sorted_items = sort_coo(tfidf.tocoo())
keywords = extract_topn_from_vector(vocab,sorted_items, 50)
print(keywords)
print(random.sample(vectorizer.stop_words_, 15))

{'pierde': 0.914, 'al': 0.911, '|': 0.907, '!': 0.736, 'gana': 0.9, 'del': 0.889, 'agua': 0.757, '...': 0.831, '-': 0.814, 'aquí': 0.751, 'inversionistas': 0.79, '4': 0.786, 'tres': 0.775, 'sin': 0.765, 'voto': 0.729, 'estaba': 0.755, '😂': 0.717, 'construido': 0.738, 'peso': 0.729, 'cancelación': 0.728, '$': 0.724, '"': 0.71, 'cómo': 0.72, '🤔': 0.719, 'unam': 0.717, 'algunos': 0.709, 'voy': 0.708, 'tema': 0.707, 'haga': 0.706, 'todos': 0.698, 'cancela': 0.696}
['lecho', 'siga', 'demostrar', 'dan', 'recorrer', 'centímetros', 'sudado', 'apoyar', 'conoce', 'conjunta', 'ganadora', 'fox', 'buena', 'acuiferos', 'https://t.co/5m8vnrkhfa']


## Parte 4: Predicción de nuevos tuits utilizando algoritmos de Machine Learning

In [4]:
rl = LR(solver='liblinear', multi_class='ovr')
rl.fit(tfidf, y_train)

pred = rl.predict(tfidf_test)

score = [metrics.accuracy_score(y_test, pred), metrics.precision_score(y_test, pred, average='macro'), 
         metrics.recall_score(y_test, pred, average='macro'), metrics.f1_score(y_test, pred, average='micro')]
print(score)

[0.729957805907173, 0.7050753595647962, 0.6628998000999501, 0.7299578059071729]


# ¡Felicidades! Acabas de completar tu primer análisis de sentimientos. Un clasificador con alrededor del 70% de éxito es excelente para los problemas de Procesamiento de Lenguaje Natural de hoy en día, sobre todo en español.

## Piensa que has sacado 70 en un examen, en el que el profe que da esa materia a lo mucho llega a dar 90's, pero solo a esos que tienen 8 GPU's y que solo se dedican a pasar esa materia.